---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    train.csv - the training set (all tickets issued 2004-2011)
    test.csv - the test set (all tickets issued 2012-2016)
    addresses.csv & latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [150]:
import pandas as pd
import numpy as np

def blight_model():
    from sklearn.model_selection import GridSearchCV
    from sklearn.ensemble import GradientBoostingClassifier

        #read the data
    df_train = pd.read_csv("train.csv",encoding="ISO-8859-1")
    df_test = pd.read_csv("test.csv", encoding="ISO-8859-1")
    lat_data = pd.read_csv("latlons.csv")
    addresses = pd.read_csv("addresses.csv")

        #Create and clean the train data
    df_train = pd.merge(df_train,pd.merge(addresses,lat_data))
    df_test = pd.merge(df_test,pd.merge(addresses,lat_data))
    df_train.set_index("ticket_id",inplace=True)
    df_test.set_index("ticket_id",inplace=True)
    df_train.dropna(subset=['compliance'],inplace=True)
    df_train["compliance"] = df_train["compliance"].astype(int)

        #Replacing NaN values for "NA" in categorical variables, and 0 for numerical variables; Converting categorical values to integer values  
    for df in [df_train,df_test]:
        for column in df.columns:
            if df[column].dtype == "object":
                df[column].fillna("NA",inplace=True)
                df[column] = df[column].astype("category").cat.codes
            else:
                df[column].fillna(0, inplace=True)

        #Selecting the columns to used
    common_cols_to_drop = ['agency_name', 'inspector_name', 'mailing_address_str_number',
                               'violator_name', 'violation_street_number', 'violation_street_name',
                               'mailing_address_str_name', 'address', 'admin_fee', 'violation_zip_code',
                               'state_fee', 'late_fee', 'ticket_issued_date', 'hearing_date', 'violation_description',
                               'fine_amount', 'clean_up_cost', 'disposition', 'grafitti_status',
                               'violation_code', 'city']
    train_columns_to_drop = [
                                'payment_status', 'payment_date', 
                                'balance_due', 'payment_amount'
        ]
    df_train.drop(common_cols_to_drop + train_columns_to_drop, inplace=True, axis=1)
    df_test.drop(common_cols_to_drop, inplace=True, axis=1)

    #Creating the X and y
    y_train = df_train["compliance"]
    X_train = df_train.drop(["compliance",'compliance_detail', 'collection_status'], axis=1)

    #Creating the model
    clf = GradientBoostingClassifier(random_state = 0)
    clf.fit(X_train, y_train)
    probs = clf.predict_proba(df_test)
    result = pd.Series(probs[:, 1], index=df_test.index)
    return result
#blight_model()

/opt/conda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2827: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


ticket_id
284932    0.100187
285362    0.039853
285361    0.106854
285338    0.130575
285346    0.140931
285345    0.130575
285347    0.135899
285342    0.365697
285530    0.177870
284989    0.042626
285344    0.125892
285343    0.069933
285340    0.070241
285341    0.137148
285349    0.110957
285348    0.100793
284991    0.069413
285532    0.087024
285406    0.055172
285001    0.130506
285006    0.100215
285405    0.025808
285337    0.101367
285496    0.100848
285497    0.110955
285378    0.027302
285589    0.051587
285585    0.094874
285501    0.118486
285581    0.040908
            ...   
376367    0.056518
376366    0.052142
376362    0.053119
376363    0.100702
376365    0.058170
376364    0.053674
376228    0.091597
376265    0.054890
376286    0.418724
376320    0.055859
376314    0.050033
376327    0.505030
376385    0.458634
376435    0.138476
376370    0.485135
376434    0.100848
376459    0.117638
376478    0.031638
376473    0.055423
376484    0.084829
376482    0.057583
37

# WORK SPACE

In [77]:
df_train.tail()

,ticket_id,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,...,balance_due,payment_date,payment_status,collection_status,grafitti_status,compliance_detail,compliance,address,lat,lon
250278,285106,1,158,47066,20009.0,1187,0.0,18715.0,12935,873,...,250.0,2307,0,1,0,7,0,37335,42.440228,-83.154829
250287,284650,1,75,31094,15725.0,1495,0.0,0.0,16757,873,...,1130.0,2307,0,0,0,7,0,20295,42.406293,-83.171850
250288,285125,1,9,13557,7152.0,318,0.0,7152.0,24784,873,...,580.0,2307,0,1,0,7,0,61335,42.366529,-83.141897
250290,284881,1,62,36255,17403.0,1151,0.0,1743.0,13790,873,...,0.0,1723,1,1,0,3,1,26490,42.422081,-83.038656
250293,284333,1,52,2686,15634.0,1194,0.0,72.0,12773,545,...,250.0,2307,0,1,0,7,0,19954,42.438867,-82.959888


In [143]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
    
    #read the data
df_train = pd.read_csv("train.csv",encoding="ISO-8859-1")
df_test = pd.read_csv("test.csv", encoding="ISO-8859-1")
lat_data = pd.read_csv("latlons.csv")
addresses = pd.read_csv("addresses.csv")
    
    #Create and clean the train data
df_train = pd.merge(df_train,pd.merge(addresses,lat_data))
df_test = pd.merge(df_test,pd.merge(addresses,lat_data))
df_train.set_index("ticket_id",inplace=True)
df_test.set_index("ticket_id",inplace=True)
df_train.dropna(subset=['compliance'],inplace=True)
df_train["compliance"] = df_train["compliance"].astype(int)
    
    #Replacing NaN values for "NA" in categorical variables, and 0 for numerical variables; Converting categorical values to integer values  
for df in [df_train,df_test]:
    for column in df.columns:
        if df[column].dtype == "object":
            df[column].fillna("NA",inplace=True)
            df[column] = df[column].astype("category").cat.codes
        else:
            df[column].fillna(0, inplace=True)

    #Selecting the columns to used
common_cols_to_drop = ['agency_name', 'inspector_name', 'mailing_address_str_number',
                           'violator_name', 'violation_street_number', 'violation_street_name',
                           'mailing_address_str_name', 'address', 'admin_fee', 'violation_zip_code',
                           'state_fee', 'late_fee', 'ticket_issued_date', 'hearing_date', 'violation_description',
                           'fine_amount', 'clean_up_cost', 'disposition', 'grafitti_status',
                           'violation_code', 'city']
train_columns_to_drop = [
                            'payment_status', 'payment_date', 
                            'balance_due', 'payment_amount'
    ]
df_train.drop(common_cols_to_drop + train_columns_to_drop, inplace=True, axis=1)
df_test.drop(common_cols_to_drop, inplace=True, axis=1)
    
    #Creating the X and y
y_train = df_train["compliance"]
X_train = df_train.drop(["compliance",'compliance_detail', 'collection_status'], axis=1)
    
clf = GradientBoostingClassifier(random_state = 0)
clf.fit(X_train, y_train)
probs = grid_clf_auc.predict_proba(df_test)[:, 1]
result = pd.Series(probs, index=df_test.index)

/opt/conda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [148]:
probs = clf.predict_proba(df_test)[:, 1]
result = pd.Series(probs, index=df_test.index)
result

ticket_id
284932    0.100187
285362    0.039853
285361    0.106854
285338    0.130575
285346    0.140931
285345    0.130575
285347    0.135899
285342    0.365697
285530    0.177870
284989    0.042626
285344    0.125892
285343    0.069933
285340    0.070241
285341    0.137148
285349    0.110957
285348    0.100793
284991    0.069413
285532    0.087024
285406    0.055172
285001    0.130506
285006    0.100215
285405    0.025808
285337    0.101367
285496    0.100848
285497    0.110955
285378    0.027302
285589    0.051587
285585    0.094874
285501    0.118486
285581    0.040908
            ...   
376367    0.056518
376366    0.052142
376362    0.053119
376363    0.100702
376365    0.058170
376364    0.053674
376228    0.091597
376265    0.054890
376286    0.418724
376320    0.055859
376314    0.050033
376327    0.505030
376385    0.458634
376435    0.138476
376370    0.485135
376434    0.100848
376459    0.117638
376478    0.031638
376473    0.055423
376484    0.084829
376482    0.057583
37

In [29]:
df_train["compliance"].notnull().sum()

159880

In [30]:
df_test = pd.read_csv("test.csv", encoding="ISO-8859-1")
df_test.head()

,ticket_id,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,...,violation_description,disposition,fine_amount,admin_fee,state_fee,late_fee,discount_amount,clean_up_cost,judgment_amount,grafitti_status
0,284932,Department of Public Works,"Granberry, Aisha B","FLUELLEN, JOHN A",10041.0,ROSEBERRY,NaN,141,ROSEBERRY,DETROIT,...,Failure to secure City or Private solid waste ...,Responsible by Default,200.0,20.0,10.0,20.0,0.0,0.0,250.0,NaN
1,285362,Department of Public Works,"Lusk, Gertrina","WHIGHAM, THELMA",18520.0,EVERGREEN,NaN,19136,GLASTONBURY,DETROIT,...,Allowing bulk solid waste to lie or accumulate...,Responsible by Default,1000.0,20.0,10.0,100.0,0.0,0.0,1130.0,NaN
2,285361,Department of Public Works,"Lusk, Gertrina","WHIGHAM, THELMA",18520.0,EVERGREEN,NaN,19136,GLASTONBURY,DETROIT,...,Improper placement of Courville container betw...,Responsible by Default,100.0,20.0,10.0,10.0,0.0,0.0,140.0,NaN
3,285338,Department of Public Works,"Talbert, Reginald","HARABEDIEN, POPKIN",1835.0,CENTRAL,NaN,2246,NELSON,WOODHAVEN,...,Allowing bulk solid waste to lie or accumulate...,Responsible by Default,200.0,20.0,10.0,20.0,0.0,0.0,250.0,NaN
4,285346,Department of Public Works,"Talbert, Reginald","CORBELL, STANLEY",1700.0,CENTRAL,NaN,3435,MUNGER,LIVONIA,...,Violation of time limit for approved container...,Responsible by Default,100.0,20.0,10.0,10.0,0.0,0.0,140.0,NaN


In [31]:
addresses = pd.read_csv("addresses.csv")
addresses.shape

(311307, 2)

In [32]:
addresses.head()

,ticket_id,address
0,22056,"2900 tyler, Detroit MI"
1,27586,"4311 central, Detroit MI"
2,22062,"1449 longfellow, Detroit MI"
3,22084,"1441 longfellow, Detroit MI"
4,22093,"2449 churchill, Detroit MI"


In [33]:
lat_data = pd.read_csv("latlons.csv")
lat_data.head()

,address,lat,lon
0,"4300 rosa parks blvd, Detroit MI 48208",42.346169,-83.079962
1,"14512 sussex, Detroit MI",42.394657,-83.194265
2,"3456 garland, Detroit MI",42.373779,-82.986228
3,"5787 wayburn, Detroit MI",42.403342,-82.957805
4,"5766 haverhill, Detroit MI",42.407255,-82.946295


In [34]:
df_train = pd.merge(df_train,pd.merge(addresses,lat_data))
df_train.head()

,ticket_id,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,...,balance_due,payment_date,payment_status,collection_status,grafitti_status,compliance_detail,compliance,address,lat,lon
0,22056,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","INVESTMENT INC., MIDWEST MORTGAGE",2900.0,TYLER,NaN,3.0,S. WICKER,CHICAGO,...,305.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0.0,"2900 tyler, Detroit MI",42.390729,-83.124268
1,27586,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Michigan, Covenant House",4311.0,CENTRAL,NaN,2959.0,Martin Luther King,Detroit,...,75.0,2005-06-02 00:00:00,PAID IN FULL,NaN,NaN,compliant by late payment within 1 month,1.0,"4311 central, Detroit MI",42.326937,-83.135118
2,22062,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","SANDERS, DERRON",1449.0,LONGFELLOW,NaN,23658.0,P.O. BOX,DETROIT,...,0.0,NaN,NO PAYMENT APPLIED,NaN,NaN,not responsible by disposition,NaN,"1449 longfellow, Detroit MI",42.380516,-83.096069
3,22084,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","MOROSI, MIKE",1441.0,LONGFELLOW,NaN,5.0,ST. CLAIR,DETROIT,...,0.0,NaN,NO PAYMENT APPLIED,NaN,NaN,not responsible by disposition,NaN,"1441 longfellow, Detroit MI",42.380570,-83.095919
4,22093,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","NATHANIEL, NEAL",2449.0,CHURCHILL,NaN,7449.0,CHURCHILL,DETROIT,...,0.0,NaN,NO PAYMENT APPLIED,NaN,NaN,not responsible by disposition,NaN,"2449 churchill, Detroit MI",42.145257,-83.208233


In [38]:
df_train.dropna(subset=['compliance'],inplace=True)
df_train.shape

(159880, 37)

In [42]:
df_train["compliance"] = df_train["compliance"].astype(int)
df_train.head()

,ticket_id,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,...,balance_due,payment_date,payment_status,collection_status,grafitti_status,compliance_detail,compliance,address,lat,lon
0,22056,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","INVESTMENT INC., MIDWEST MORTGAGE",2900.0,TYLER,NaN,3.0,S. WICKER,CHICAGO,...,305.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0,"2900 tyler, Detroit MI",42.390729,-83.124268
1,27586,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Michigan, Covenant House",4311.0,CENTRAL,NaN,2959.0,Martin Luther King,Detroit,...,75.0,2005-06-02 00:00:00,PAID IN FULL,NaN,NaN,compliant by late payment within 1 month,1,"4311 central, Detroit MI",42.326937,-83.135118
5,22046,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","KASIMU, UKWELI",6478.0,NORTHFIELD,NaN,2755.0,E. 17TH,LOG BEACH,...,305.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0,"6478 northfield, Detroit MI",42.145257,-83.208233
6,18738,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Deerwood Development Group Inc, Deer",8027.0,BRENTWOOD,NaN,476.0,Garfield,Clinton,...,855.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0,"8027 brentwood, Detroit MI",42.433466,-83.023493
7,18735,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Rafee Auto Services L.L.C., RAF",8228.0,MT ELLIOTT,NaN,8228.0,Mt. Elliott,Detroit,...,140.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0,"8228 mt elliott, Detroit MI",42.388641,-83.037858
